## P1 - PREPROCESAMIENTO

In [59]:
import nltk
import re

# Cargar el archivo de stoplist [CON ESTE TRABAJAREMOS countwords]
with open('stoplist.txt', 'r') as file:
    stoplist = file.read().splitlines()
stoplist += ['.', ',', ';', ':', '!', '?', '¿', '¡', '(', ')', '[', ']', '{', '}', '"', "'"]

def filter_stoplist(text):
    return ' '.join([word.lower() for word in text.split() if word not in stoplist])

# a) Filtrar los stop words usando el stoplist.txt
for i in range(1, 7):
    with open(f'libro{i}.txt', 'r') as file:
        text = file.read()
        text = re.sub(r'[.,;:!?¿¡()\[\]{}"\'-]', '', text) # b) Retirar los signos innecesarios
        text = text.lower() # convertir a minúsculas para el stemming
        text = filter_stoplist(text)
        
        with open(f'libro{i}_filtered.txt', 'w') as file:
            file.write(text)
            
# b) Reemplazar cada palabra por su raíz (stemming)
from nltk.stem import SnowballStemmer

lexema = SnowballStemmer('spanish')

for i in range(1, 7):
    with open(f'libro{i}_filtered.txt', 'r') as file:
        text = file.read()
        text = ' '.join([lexema.stem(word) for word in text.split()])
        with open(f'libro{i}_stemmed.txt', 'w') as file:
            file.write(text)
            
# guardar en listas las palabras de cada texto
texts = []
for i in range(1, 7):
    with open(f'libro{i}_stemmed.txt', 'r') as file:
        texts.append(file.read().split())

print('10 PRIMEROS LEXEMAS [Libro 1]: ',texts[0][0:10]) 

texts2 = []
for i in range(1, 7):
    with open(f'libro{i}_filtered.txt', 'r') as file:
        texts2.append(file.read().split())
        
print('10 PRIMERAS PALABRAS [Libro 1]: ',texts2[0][0:10])

10 PRIMEROS LEXEMAS [Libro 1]:  ['obra', 'comienz', 'notici', 'celebraciã³n', '111âº', 'cumpleaã±', 'bilb', 'bolsã³n', 'comarc', 'bilb']
10 PRIMERAS PALABRAS [Libro 1]:  ['obra', 'comienza', 'noticia', 'celebraciã³n', '111âº', 'cumpleaã±os', 'bilbo', 'bolsã³n', 'comarca', 'bilbo']


## P2 - CONSTRUCCIÓN DE ÍNDICE INVERTIDO

In [61]:
# contar la frecuencia de cada lexema
from collections import Counter

# contar la frecuencia de cada palabra   
word_freq = Counter()
for text in texts2:
    word_freq.update(text)
    
#print('FRECUENCIA DE PALABRAS: ',word_freq)
        
# seleccionar las 500 palabras más frecuentes
most_commom_words = word_freq.most_common(500)
most_commom_words = [word for word, freq_word in most_commom_words]

# a) construir el índice invertido [palabras]
index_words = {}
for i, textword in enumerate(texts2, start=1):
    word_freq = {}
    for word1 in textword:
        if word1 in most_commom_words:
            word_freq[word1] = word_freq.get(word1, 0) + 1
    for word1, freq_word in word_freq.items():
        if word1 not in index_words:
            index_words[word1] = []
        index_words[word1].append((i, freq_word))
            
# b) guardar el índice en un archivo de texto[PALABRAS]
with open('index_words.txt', 'w') as file:
    for word1 in sorted(index_words.keys()): # ordenar el índice alfabéticamente
        file.write(f'{word1}: {", ".join([f"{i},{j}" for i, j in index_words[word1]])}\n')
        
# leer el archivo de texto
with open('index_words.txt', 'r') as file:
    print("Muestra de índice invertido:\n",file.read()[10:111]) # mostrar entre 11 a 111 caracteres

Muestra de índice invertido:
 
abismo: 2,1, 3,2
acaba: 1,1, 4,2, 5,1
acababa: 1,1
acaban: 1,1, 6,1
acabã³: 1,1
accidentalmente: 1,1


## P3 - APLICAR CONSULTAS BOOLEANAS

In [62]:
list_libros = [1, 2, 3, 4, 5, 6] # libros del 1 al 6

def L(word): # busca en que libros se encuentra la palabra
    result = []
    for i, text in enumerate(texts2, start=1):
        if word in text:
            result.append(i)
    return result

def AND(A, B): # retorna los libros en los que se encuentran ambas palabras
    i, j = 0, 0
    result = []
    while i < len(A) and j < len(B):
        if A[i] == B[j]:
            result.append(A[i])
            i += 1
            j += 1
        elif A[i] < B[j]:
            i += 1
        else:
            j += 1
    return result # lo mismo que: [i for i in A if i in B]


def OR(A, B): # retorna los libros en los que se encuentran al menos una de las palabras
    i, j = 0, 0
    result = []
    while i < len(A) and j < len(B):
        if A[i] == B[j]:
            result.append(A[i])
            i += 1
            j += 1
        elif A[i] < B[j]:
            result.append(A[i])
            i += 1
        else:
            result.append(B[j])
            j += 1
    while i < len(A):
        result.append(A[i])
        i += 1
    while j < len(B):
        result.append(B[j])
        j += 1
    return result # lo mismo que A + list(set(B) - set(A)), pero en orden

def NOT(A): # retorna los libros en los que no se encuentra la palabra
    lista = [i for i in list_libros if i not in A]
    return lista

def ANDNOT(A,B): # retorna los libros en los que se encuentra la primera palabra pero no la segunda
    i, j = 0, 0
    result = []
    while i < len(A) and j < len(B):
        if A[i] == B[j]:
            i += 1
            j += 1
        elif A[i] < B[j]:
            result.append(A[i])
            i += 1
        else:
            j += 1
    while i < len(A):
        result.append(A[i])
        i += 1
    return result #lo mismo que [i for i in A if i not in B] pero en orden

# Ejecución
result1 = AND(L('comunidad'), L('frodo'))
result2 = OR(L('comunidad'), L('frodo'))
result3 = NOT(L('comunidad'))
result4 = ANDNOT(L('frodo'), L('gondor'))
print("Comunidad en los libros: ", L('comunidad'))
print("Frodo en los libros: ", L('frodo'))
print("Gondor en los libros: ", L('gondor'))
print("")
print(f'comunidad AND frodo: {result1}') # INTERSECCION
print(f'comunidad OR frodo: {result2}') # UNION
print(f'NOT comunidad: {result3}') # NEGACION
print(f'frodo AND-NOT gnndor: {result4}') # DIFERENCIA

print()
print('Ejemplo de consultas:\n')

#b) Probar el programa con al menos 3 consultas y al menos 3 términos
consulta1 = ANDNOT(AND(L('comunidad'), L('frodo')), L('gondor')) # [2] - [2,3,5,6] = []
print(f'(comunidad AND frodo) AND-NOT gondor: {consulta1}')

consulta2 = OR(AND(L('comunidad'), L('frodo')), L('gondor')) # [2] + [2,3,5,6] = [2,3,5,6]
print(f'(comunidad AND frodo) OR gondor: {consulta2}')

#usar las palabras gandalf, hermana y gracias
consulta3 = OR(ANDNOT(L('gandalf'), L('hermana')), L('gracias')) # ([1,2,3,5,6] - [5,6]) + [1,4,5] = [1,2,3] + [1,4,5] = [1,2,3,4,5]
print(f'(gandalf AND-NOT hermana) OR gracias: {consulta3}')

Comunidad en los libros:  [2]
Frodo en los libros:  [1, 2, 3, 4, 5, 6]
Gondor en los libros:  [2, 3, 5, 6]

comunidad AND frodo: [2]
comunidad OR frodo: [1, 2, 3, 4, 5, 6]
NOT comunidad: [1, 3, 4, 5, 6]
frodo AND-NOT gnndor: [1, 4]

Ejemplo de consultas:

(comunidad AND frodo) AND-NOT gondor: []
(comunidad AND frodo) OR gondor: [2, 3, 5, 6]
(gandalf AND-NOT hermana) OR gracias: [1, 2, 3, 4, 5]
